<a href="https://colab.research.google.com/github/Sharon-Tseng/VincentMobile_Database_System/blob/main/ISOM3260_Progress_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer and Member

In [4]:
# -- Copy and paste the whole code when doing project--
# -- Install and setup Oracle Instant Client --
! wget -O oracleinstantclient.zip https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
! unzip -oq oracleinstantclient.zip
! mkdir -p /opt/oracle
! mv instantclient_19_6 /opt/oracle/
! sh -c "echo /opt/oracle/instantclient_19_6 > /etc/ld.so.conf.d/oracle-instantclient.conf"
# create a link
! ldconfig


# -- Include cx_Oracle package --
! pip install cx_Oracle

--2025-03-28 16:50:03--  https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
Resolving download.oracle.com (download.oracle.com)... 23.193.24.98
Connecting to download.oracle.com (download.oracle.com)|23.193.24.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75462547 (72M) [application/zip]
Saving to: ‘oracleinstantclient.zip’

oracleinstantclient 100%[===================>]  71.97M   116MB/s    in 0.6s    

2025-03-28 16:50:04 (116 MB/s) - ‘oracleinstantclient.zip’ saved [75462547/75462547]

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: 

In [5]:
# -- Copy and paste the whole code when doing project, using personal account for testing --
# Import cx_Oracle package and connect to Oracle Database
import cx_Oracle

HOST_NAME = " IMZ409.ust.hk"
PORT_NUMBER = "1521"
SERVICE_NAME = "imz409"
USERNAME = "ytsengaa"  # use your own username and password
PASSWORD = "7192"    # use your own username and password

dsn_tns = cx_Oracle.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME)
conn = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns)

c = conn.cursor()   # c: represent database connection

In [6]:
from google.colab.widgets import TabBar
import ipywidgets as widgets
import time
import pandas as pd
from ipywidgets import interact
from ipywidgets import Text
from ipywidgets import DatePicker
from ipywidgets import IntText
from ipywidgets import Label
from datetime import date

In [10]:
from pickle import FALSE
# tab object
tb_all = TabBar(['Introduction','Register','Login','Welcome','Forget_Password','Message_Box_1','Message_Box_2'])

# a simple function for jumping between tabs
def showPage(tabName):
  with tb_all.output_to(tabName):
    display()

##############################################
############## Introduction Tab ###########
##############################################

with tb_all.output_to('Introduction'):
  intro_label = Label("Welcome to 302 Vincent Mobile")

  btn_RegisterPage = widgets.Button(description="Register as Member")
  btn_LoginPage = widgets.Button(description="Login")

  def on_btnRegisterPage_button_clicked(b):
    showPage("Register")
  btn_RegisterPage.on_click(on_btnRegisterPage_button_clicked)

  def on_btn_LoginPage_button_clicked(b):
    showPage("Login")
  btn_LoginPage.on_click(on_btn_LoginPage_button_clicked)

  intro_row1HBox = widgets.HBox([intro_label])
  intro_row2HBox = widgets.HBox([btn_LoginPage])
  intro_row3HBox = widgets.HBox([btn_RegisterPage])

  display(widgets.VBox([intro_row1HBox, intro_row2HBox, intro_row3HBox]))

##############################################
############## Register Tab ###############
##############################################

with tb_all.output_to("Register"):
  reg_label = Label("Please fill in the information")
  username_label = Label("Username: ")
  username_textbox = Text(placeholder='e.g, achencd', disabled=False)
  password_label = Label("Password: ")
  password_textbox = Text(placeholder='e.g, 123rteyq!', disabled=False)
  firstname_label = Label("First Name: ")
  firstname_textbox = Text(placeholder='Tai Ming', disabled=False)
  lastname_label = Label("Last Name: ")
  lastname_textbox = Text(placeholder='Wong', disabled=False)
  address_label = Label("Address: ")
  address_textbox = Text(placeholder='Sai Kung, Hong Kong', disabled=False)
  gender_label = Label("Gender: ")
  gender_dropdown = widgets.Dropdown(options=["Male", "Female", "Others"])
  dob_label = Label("Date of Birth: ")
  dob_datepicker = DatePicker(placeholder='2000/01/01', disabled=False)
  phone_label = Label("Phone: ")
  phone_textbox = Text(placeholder='12345678', disabled=False)
  email_label = Label("Email: ")
  email_textbox = Text(placeholder='e.g, abc@defg', disabled=False)
  creditno_label = Label("Credit No.: ")
  creditno_textbox = Text(placeholder='1234 5678 9123 4567', disabled=False)
  security_question_label = Label("Security Question: ")
  security_question_textbox = Text(placeholder='Security Question', disabled=False)
  security_answer_label = Label("Security Answer: ")
  security_answer_textbox = Text(placeholder='Security Answer', disabled=False)
  expiry_date_label = Label("Expiry Date: ")
  expiry_date_datepicker = DatePicker(dplaceholder='1991-01-01', disabled=False)
  cvc_label = Label("CVC: ")
  cvc_textbox = Text(placeholder='000', disabled=False)

  reg_register_button = widgets.Button(description="Register")
  reg_back_button = widgets.Button(description="Back")

  def on_register_button_clicked(b):
    c.execute(
    """
    INSERT INTO MEMBER VALUES(seq_member.nextval, '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}' ,TO_DATE('{}', 'YYYY-MM-DD'), '{}', '{}', '{}', TO_DATE('{}', 'YYYY-MM-DD'), '{}')
    """.format(firstname_textbox.value, lastname_textbox.value,
          username_textbox.value, password_textbox.value, address_textbox.value,
          security_question_textbox.value, security_answer_textbox.value, creditno_textbox.value,
          expiry_date_datepicker.value, cvc_textbox.value, phone_textbox.value, email_textbox.value,
          dob_datepicker.value, gender_dropdown.value)
    )

    c.execute("commit")
    reg_register_button.disabled=True
    msg_label_1.value = "Successfully Registered!"
    showPage("Message_Box_1")
  reg_register_button.on_click(on_register_button_clicked)

  def on_back_button_clicked(b):
    showPage("Introduction")
  reg_back_button.on_click(on_back_button_clicked)

  # display objects
  reg_msg_labelVBox = widgets.VBox([reg_label])
  reg_labelVBox = widgets.VBox([username_label, password_label, firstname_label, lastname_label,
                  address_label,gender_label,dob_label,phone_label,email_label,
                  creditno_label,security_question_label,security_answer_label,expiry_date_label,cvc_label])
  reg_textboxVBox = widgets.VBox([username_textbox, password_textbox, firstname_textbox, lastname_textbox,
                  address_textbox,gender_dropdown,dob_datepicker,phone_textbox,email_textbox,
                  creditno_textbox,security_question_textbox,security_answer_textbox,expiry_date_datepicker,cvc_textbox])
  reg_line1HBox = widgets.HBox([reg_msg_labelVBox])
  reg_line2HBox = widgets.HBox([reg_labelVBox, reg_textboxVBox])
  reg_line3HBox = widgets.HBox([reg_register_button, reg_back_button])

  display(widgets.VBox([reg_line1HBox,reg_line2HBox, reg_line3HBox]))

##############################################
############## Login Tab ##################
##############################################

with tb_all.output_to('Login'):
  login_label = Label("Please login with your username and password:")
  login_un_label = Label("Username: ")
  login_un_textbox = Text(placeholder='e.g, achencd', disabled=False)
  login_pwd_label = Label("Password: ")
  login_pwd_textbox = Text(placeholder='e.g, 123rteyq!', disabled=False)

  login_button = widgets.Button(description="Login")
  forget_pwd_button = widgets.Button(description="Forget Password")
  login_back_button = widgets.Button(description="Back")

  def on_button_login_clicked(b):

    c.execute(
    """
    SELECT M_FIRST_NAME, M_LAST_NAME
    FROM MEMBER
    WHERE EXISTS (SELECT M_USERNAME, M_PW FROM MEMBER WHERE M_USERNAME = '{}' AND M_PW = '{}')
    """.format(login_un_textbox.value, login_pwd_textbox.value)
    )

    login_un_textbox.value = ""
    login_pwd_textbox.value = ""

    if c.fetchone():
      msg_label_2.value = "Welcome back!"
      showPage("Message_Box_2")
    else:
      msg_label_1.value = "Incorrect username or password! Please try again."
      showPage("Message_Box_1")

  login_button.on_click(on_button_login_clicked)

  def on_back_button_clicked(b):
    showPage("Introduction")
  login_back_button.on_click(on_back_button_clicked)

  # design login interface
  login_msg_labelVBox = widgets.VBox([login_label])
  login_labelVBox = widgets.VBox([login_un_label,login_pwd_label])
  login_textboxVBox = widgets.VBox([login_un_textbox,login_pwd_textbox])

  login_line1HBox = widgets.HBox([login_msg_labelVBox])
  login_line2HBox = widgets.HBox([login_labelVBox, login_textboxVBox])
  login_line3HBox = widgets.HBox([login_button, forget_pwd_button, login_back_button])

  display(widgets.VBox([login_line1HBox, login_line2HBox, login_line3HBox]))

  def on_forget_pwd_clicked(b):
    showPage("Forget_Password")
  forget_pwd_button.on_click(on_forget_pwd_clicked)

# with tb_all.output_to('Forget_Password'):




##############################################
############## Message Box ################
##############################################

## Message_Box_1 ##
with tb_all.output_to('Message_Box_1'):
  msg_label_1 = Label("")
  msg_ok_btn = widgets.Button(description="OK")

  def on_msgbox_click(b):
    showPage("Introduction")
  msg_ok_btn.on_click(on_msgbox_click)

  msg_row1HBox = widgets.HBox([msg_label_1])
  msg_row2HBox = widgets.HBox([msg_ok_btn])

  display(widgets.VBox([msg_row1HBox, msg_row2HBox]))

## Message_Box_2 ##
with tb_all.output_to('Message_Box_2'):
  msg_label_2 = Label("")
  msg_ok_btn = widgets.Button(description="OK")

  def on_msgbox_click(b):
    showPage("Welcome")
  msg_ok_btn.on_click(on_msgbox_click)

  msg_row1HBox = widgets.HBox([msg_label_2])
  msg_row2HBox = widgets.HBox([msg_ok_btn])

  display(widgets.VBox([msg_row1HBox, msg_row2HBox]))

showPage("Introduction")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>